<h1 align = center> <b> Reinforcement Learning (M2 MOSIG / MSIAM)<h1>
<h2 align = center> <strong> Lab 3: Non-stochastic Multi-armed Bandits: Hedge & Exp3 <h2>   

## Introduction

In this TP, we conduct several experiments on several regret-minimization algorithms in Multi-armed Bandit Problems. This notebook contains 2 main sections concerning 2 main feedback set-up: **full-information** and **bandit feedback**; in which, we study two algorithms, HEDGE and EXP3, which are variants of the Exponential Weights algorithmic template. 

$
%----------------------------------------------------------------------
%% Modifiers
%----------------------------------------------------------------------
%\newcommand{\alt}[1]{#1'}		% for alternates
\newcommand{\bb}[1]{\mathbf{#1}}		% for bold
%
%
%----------------------------------------------------------------------
%% Number fields
%----------------------------------------------------------------------
\newcommand{\F}{\mathbb{F}}		% generic field
\newcommand{\N}{\mathbb{N}}		% for naturals
\newcommand{\Z}{\mathbb{Z}}		% for integers
\newcommand{\Q}{\mathbb{Q}}		% for rationals
\newcommand{\R}{\mathbb{R}}		% for reals
\newcommand{\CC}{\mathbb{C}}		% for complex numbers (may clash)
%----------------------------------------------------------------------
%% Operators
%----------------------------------------------------------------------
\DeclareMathOperator{\bigoh}{\mathcal O}		% for Landau O
\DeclareMathOperator{\ess}{ess}		% for essential
\DeclareMathOperator{\grad}{\nabla}		% for gradient
\DeclareMathOperator{\Hess}{Hess}		% for Hessian
\DeclareMathOperator{\Jac}{Jac}		% for Hessian
\DeclareMathOperator{\ind}{ind}		% for index
\DeclareMathOperator{\rank}{rank}		% for rank
\DeclareMathOperator{\sign}{sgn}		% for sign
\DeclareMathOperator{\Sym}{Sym}		% for symmetric
\newcommand{\inflow}{r}
\newcommand{\socialcost}{\mathfrak{C}}
\newcommand{\opt}{\texttt{opt}}
\newcommand{\NEs}{\texttt{NE}}
\newcommand{\poa}{\textrm{PoA}}
\newcommand{\pos}{\textrm{PoS}}
$

### Preliminary Libraries: This notebook requires the following libraries: NumPy, Matplotlib and Pandas. 

If you want to run this notebook in your local machine, you can install them with the following command:

``
pip install numpy matplotlib
``

If you use the Google Colab environment, you can you run the next cell to check if these libraries are already installed; if not, install them on the server machine.


In [ ]:
colab_requirements = ["numpy","matplotlib"]
import sys, subprocess
def run_subprocess_command(cmd):
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    for line in process.stdout:   print(line.decode().strip())
        
if "google.colab" in sys.modules:
    for i in colab_requirements:
        run_subprocess_command("pip install " + i)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
import copy

# Section 0. Problem Set-up

### 0.1. The game:

In this session, we focus on the decision-making process of a **learner** (a decision-maker) who has the action set $\mathcal{A}$ containing $m$ actions (labeled $0, \ldots, m$):

----
**For** $n=1,2,\ldots, T$, **do**:
* Choose a vector $x_n \in \Delta(\mathcal{A})$ (i.e., a mixed strategy) indicating the probabily of choosing each action.  
* Draw and play an action $a_n \sim x_n$
* Encouter a payoff vectors $v_n \in \mathbb{R}^{m}$ and receives a payoff $v_{n,a_n}$.
* Observe a feedback on $v_n$

---

### 0.2. Payoff vectors

**Remark**: Naturally, we would like to simulate a problem set-up where payoff vectors depend on decisions of the learner and of several adversarial opponents. However, it is not useful to define a specific game; instead, **at each time $n$, we generate $v_n$ randomly** (a priori hidden from the learner). 



In the next cell, we write the code for (randomly) generating a matrix with dimension $T \times n$ such that the $t$-th row of the matrix is the pure payoff vector $v_n$.


In [ ]:
def payoff_vector_gen(T,m):
    payoff = np.empty((T,m))    # initialize payoff matrix 
    for n in range(T): # For each time epoch n
        mu = np.random.uniform(low=0, high=10, size=m)    # Choose randomly m mean 
        StdVar = np.random.uniform(low=0.0, high=2, size=m)    # Choose randomly m variance
        for a in range(m):    # For each action
            payoff[n,a] =  np.random.normal(mu[a],StdVar[a],1)  # Generate 1 random payoff from N(mu[a],StdVar[a]) & assign it to payoff matrix
    return payoff

We generate the payoff vectors for a game with $m=10$ actions and the time horizon $T= 10000$.


In [ ]:
np.random.seed(15)    # Fix a random seed so that we all have the same payoff vectors
m=10
T=10000
payoff_vec = payoff_vector_gen(T,m)
print(payoff_vec)


### 0.3. Tools to compute regret:

In the sequel, we will measure the performance of algorithms via the (realized) regret:

\begin{equation}
  \textrm{Reg} (T) = \max_{a \in \mathcal{A}} \sum_{n=1}^T \textrm{payoff}[n,a] - \textrm{payoff}[n,a_n].
\end{equation}


To compute the regret, we need to look for a **best-action in hindsight** which is simply the action maximizing the total payoff if its is used repeatedly. We do this in the next cell:

In [ ]:
best_action = 0
best_cumul_payoff = -np.Inf
for a in range(m):  # For each action
    cumul_payoff = np.sum(payoff_vec[:,a])
    if cumul_payoff > best_cumul_payoff:
        best_action = int(a)
        best_cumul_payoff = cumul_payoff

print('Best action is: ', best_action)

Finally, we define a function to compute the realized regret of a given history of play (i.e., a policy):

In [ ]:
def regret(history_play):
    regret = 0
    regret_history = []
    for n in range(T): 
        regret += float(payoff_vec[n,best_action] - payoff_vec[n,history_play[n]])
        regret_history.append(regret)
    return np.array(regret_history)

# Section 1: HEDGE algorithm: Online Learning with full, exact information

In this section, we code the HEDGE algorithm as follows:

<img src="https://raw.githubusercontent.com/dongquan-vu/teaching/main/HEDGE.PNG" style="float: left"  width= "800x" />




In order to code the ExpWeight algorithm, in the next cell, we prepare the logit-mapping $\Lambda: \mathbb{R}^m \rightarrow \mathbb{R}^m$ defined as:
\begin{equation}
  \Lambda_a(y)= \underbrace{\frac{\exp(y_a)}{\sum_{a^{\prime} \in \mathcal{A}} \exp(y_{a^{\prime}})}}_{\textrm{computer does not like this form}} = \underbrace{\frac{1}{{\sum_{a^{\prime} \in \mathcal{A}} \exp(y_{a^{\prime}} - y_a  )}}}_{\textrm{computer can handle this}}.
\end{equation}

In [ ]:
def logit_mapping(y):  #taking an np.array y as input
    output = np.zeros(m)  #initialize to store output
    for a in range(m):
        output[a] = 1/ np.sum(np.exp(y - y[a]))
    return output


**RECALL**: The expected regret of HEDGE is upper-bounded as follows:

 
$$
 \texttt{Reg}_T(\texttt{Hedge}) \le \sqrt{2 \log(|\mathcal{A}|) \cdot T} = \mathcal{O}(\sqrt{T})
$$

### QUESTION 1.1: In the next cell, we code the HEDGE algorithm:

In [ ]:
def Hedge(m,T,gamma_init=0.1):
    y = np.zeros(m) # Initialize y
    gamma = gamma_init  # Initialize step-size
    history_play = [] # To store the history of plays
    for n in range(T):  #For each time epoch n
        ##### BEGIN EXERCISE ###########
        # Step 1: Compute a mixed strategy by logit-mapping:
        x_n =  #.........
        # Step 2: Draw a pure strategy from x_n:
        a_n =  #.........# Choose a number in list(range(m)) with prob distribution x_n
        # Step 4: Feedback
        V_n =  payoff_vec[n]
        # Step 5: Update y
        y = #...................
        ######## END OF EXERCISE #######
        #Record the played action:
        history_play.append(int(a_n))
        gamma = gamma_init/(1+ np.sqrt(n))
    return history_play

### Visualize regret upper-bound of HEDGE


In the next cell, we run ``Hedge`` in several episodes and record the (realized) regret. 

In [ ]:
N_episodes = 20
sum_regret_hedge, avg_regret_hedge = np.array([0]*T),np.array([0]*T)
for episode_ind in range(N_episodes):
    print("\rCount episode ", episode_ind, end="")
    hedge_result  = Hedge(m,T,gamma_init=10e-5)
    regret_episode = regret(hedge_result)
    if episode_ind==0:
        min_regret_hedge = np.copy(regret_episode)
        max_regret_hedge = np.copy(regret_episode)
    sum_regret_hedge = sum_regret_hedge +  np.copy(regret_episode)
    min_regret_hedge = np.minimum(min_regret_hedge,regret_episode)
    max_regret_hedge = np.maximum(max_regret_hedge,regret_episode)
avg_regret_hedge = np.copy(sum_regret_hedge)/N_episodes


We then plot out the mean regret (taken averagely from these episodes) representing by the red line. The pink region shows the variance of the realized regret.


**Remark**: We use log-log plot to improve the visualibility. 

In [ ]:
# Plot out:
fig,ax = plt.subplots()
ax.plot(avg_regret_hedge, color ='red')
ax.fill_between(list(range(T)), min_regret_hedge,max_regret_hedge,  color ='red', alpha=0.3,linewidth=1)
ax.grid()

# ax.plot(sqrt_t)
ax.set_yscale('log')
ax.set_xscale('log')
ax.plot()
ax.set_ylim(10e-3,10e2)
ax.set_title('Realized Regret of HEDGE')

### Question 1.2:  Analyze the plots

* 1.2.1) From the above plot, can we know for sure the order of an upper-bound? 



* 1.2.2) Assume that ```average_regret_hedge```$ \le \mathcal{O} (n^{p})$, then what do you think would happen if we plot out: ```average_regret_hedge``` $\big/ n^{p}$ 



Write your answer here:











### Question 1.3:  Having a better plot:
In the next cell, you need to plot out the evolution of ```average_regret_hedge```   $\big/ \sqrt{n}$, what can you conclude from this plot?

In [ ]:
# Plot out:
fig,ax = plt.subplots()
t_sqrt = (1+ np.power(np.array(range(T)),0.5))


ax.plot((avg_regret_hedge) / t_sqrt, color ='red')
ax.grid()

# ax.plot(sqrt_t)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(10e-3,10e2)
ax.set_title('(Expected Regret of HEDGE)' + '$ \cdot  \sqrt{n}$' )

### Bonus question: To make sure about the order of the regret's upper-bound, 
Try to plot out ```average_regret_fi```   $\big/ n^{0.7}$ and ```average_regret_fi```   $\big/ n^{0.3}$

In [ ]:
## Answer for bonus question goes here.

# SECTION 2: EXP3 Algorithm - Online Learning with Bandit Feedback

In this section, we study the bandit feedback setting: instead of observing the whole vector $V_n$ as in previous section, the learner only observes the payoff corresponding to the played pure action $a_n$. 

In this setting, we need to run "re-construct" the payoff-vector from this partial information, via the following **Important weights estimators (IWE)**:

\begin{equation}
  \hat{v}_{a,n} = \left\{\begin{array}{lr}
        &{v_{a,n}} \big/ {x_a} &, \textrm{ if } a = a_n \textrm{(i.e., if a is drawn)}\\
        &0  &,\textrm{ otherwise } 
        \end{array}\right.
\end{equation}
<img src="https://raw.githubusercontent.com/dongquan-vu/teaching/main/EXP3.PNG" style="float: left"  width= "600x" />



**RECALL**: The expected regret of Exp3 is upper-bounded as follows:

$$
 \texttt{Reg}_T(\texttt{Exp3}) \le 2\sqrt{A\log(|\mathcal{A}|) \cdot T} = \mathcal{O}(\sqrt{|\mathcal{A}|} \cdot \sqrt{T})
$$

## Question 2.1. In the next cell, we code Exp3:

In the next cell, write the code of ExpWeights that uses IWE instead of the full exact payoff vectors. 

In [ ]:
def Exp3(m,T,gamma_init=0.01):
    y = np.zeros(m) # Initialize y
    gamma = gamma_init  # Initialize step-size
    history_play = [] # To store the history of plays
    for n in range(T):  #For each time epoch n
        ##### BEGIN EXERCISE ###########
        # Step 1: Compute a mixed strategy by logit-mapping:
        x_n = #.............. 
        # Step 2: Draw a pure strategy from x_n:
        a_n = #............... # Choose a number in list(range(m)) with prob distribution x_n
        # Step 4: Feedback via IWE:
        V_hat_n #.....................
        # Step 5: Update y
        y = #.....................
        ######## END OF EXERCISE #######
        #Record the played action:
        history_play.append(int(a_n))
        gamma = gamma_init/(1+ np.sqrt(n))
    return history_play

In the next cell, we run Exp3 in several epsiodes, record the average regret.

In [ ]:
N_episodes = 20
sum_regret_exp3, avg_regret_exp3 = np.array([0]*T),np.array([0]*T)
for episode_ind in range(N_episodes):
    print("\rCount episode ", episode_ind, end="")
    exp3_result  = Exp3(m,T,gamma_init=10e-5)
    regret_episode = regret(exp3_result)
    if episode_ind==0:
        min_regret_exp3 = np.copy(regret_episode)
        max_regret_exp3 = np.copy(regret_episode)
    sum_regret_exp3 = sum_regret_exp3 +  np.copy(regret_episode)
    min_regret_exp3 = np.minimum(min_regret_exp3,regret_episode)
    max_regret_exp3 = np.maximum(max_regret_exp3,regret_episode)
avg_regret_exp3 = np.copy(sum_regret_exp3)/N_episodes

### Question 2.2. Visualize regret upper-bound of Exp3

In the next cell, we want to see the upper-bound of the expected regret of Exp3.

From what you have seen in the case of Hedge, what should you plot out?

In [ ]:
# Plot out:
fig,ax = plt.subplots()

#Begin exercise:

# FILL IN THE BLANK IN THE COMMAND BELOW:
ax.plot(     , color ='green')

#End exercise


ax.grid()
ax.set_yscale('log')
ax.set_xscale('log')
ax.plot()
ax.set_ylim(10e-3,10e1)
ax.set_title('(Expected Regret of Exp3)' + '$ \cdot  \sqrt{n}$' )